In [9]:
import pymc as pm
import numpy as np
import arviz as az
from pymc.math import logit

%load_ext lab_black
%load_ext watermark

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Priors as hidden mixtures

This example demonstrates ...

Adapted from [unit 9: cumulative2.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/cumulative2.odc).

## Associated lecture video: Unit 9 Lesson 8

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=95" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

Use of CUMULATIVE as a check for outliers

How consistent are the data with the model? Check cumulative(y[i], y[i]).

The first y[i] specifies the distribution of y[i]'s, the second is the argument/observation. The output is empirical cdf of y[i]s evaluated at y[i].

Values close to 0 or 1 are potential outliers.

What exactly does the cumulative function in BUGS do????

In [7]:
# fmt: off
y = np.array(
    [0.0, 1.0, 2.0, -1.0, 0.4, -0.5, 0.7, -1.2, 0.1, -0.4, 
     0.2, -0.5, -1.4, 1.8, 0.2, 0.3, -0.6, 1.1, 5.1, -6.3]
)
# fmt: on

In [ ]:
with pm.Model() as m:
    theta = pm.Flat("theta")
    log_sigma = pm.Flat("log_sigma")
    tau = pm.Deterministic("tau", 1/exp(2*log_sigma))
    
    cuy = pm.Deterministic("cuy", ???cumulative)
    add = pm.Deterministic("add", logit(cuy)**2)
    a = pm.Deterministic("a", pm.math.sum(add))
    
    pm.Normal("y", mu=theta, tau=tau, observed=y)
    
    trace = pm.sample(2000)

In [ ]:
az.summary(trace)

In [10]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

arviz: 0.12.1
numpy: 1.22.4
pymc : 4.0.0

